* [Titanic 100%](https://www.kaggle.com/blurredmachine/titanic-survival-a-complete-guide-for-beginners)
* [MLCource](https://mlcourse.ai/)

In [1]:


df = pd.read_csv('train.csv')
tsd = pd.read_csv('test.csv')

NameError: name 'pd' is not defined